<a href="https://www.kaggle.com/code/jayapat/project1-titanic-python?scriptVersionId=104452441" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


Load the Train Data <br>
Load the Test Data <br>
Load the gender_submission data <br>

In [2]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
gender_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

See the first few lines of the each data sets <br>
Note that the gender submisson file is the file which is given as an example of how the output should be for the test data. <br>

In [3]:
train_data.head()
test_data.head()
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


Here the gender submission file assumes that all the females survived and all the males died, so lets find out how true is this based on the train data. Before that for the sake of simplicity, make all the columns name in the lower case.

In [4]:
train_data.columns = train_data.columns.str.lower()
test_data.columns = test_data.columns.str.lower()
gender_submission.columns = gender_submission.columns.str.lower()
print(train_data.columns)
print(test_data.columns)

Index(['passengerid', 'survived', 'pclass', 'name', 'sex', 'age', 'sibsp',
       'parch', 'ticket', 'fare', 'cabin', 'embarked'],
      dtype='object')
Index(['passengerid', 'pclass', 'name', 'sex', 'age', 'sibsp', 'parch',
       'ticket', 'fare', 'cabin', 'embarked'],
      dtype='object')


In [5]:
female_s_rate = train_data[train_data['sex'] == 'female']['survived'].value_counts(normalize=True); print('female survival rate: ',female_s_rate[1]*100)
male_s_rate = train_data[train_data['sex'] != 'female']['survived'].value_counts(normalize=True); print('male survival rate: ' ,male_s_rate[1]*100)

female survival rate:  74.20382165605095
male survival rate:  18.890814558058924


Though out initial intution is not so wrong, there is chance of improvment. Here, with the intution we have predicted the survival rate with the 74% of accuracy, lets use the advance data science techniques. lets use the rendom forest first.

In [6]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["survived"]

features = ["pclass", "sex", "sibsp", "parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.passengerid, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
